# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [54]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [55]:
# checking your current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [56]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [57]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [59]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [60]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [61]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Task 1: 
    Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Step 1: Create sessions table

In [62]:
query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(session_id text, item_in_session text, artist text, song_title text, song_length text, PRIMARY KEY (session_id, item_in_session, artist))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Step 2: Get data from event_datafile_new.csv file and insert into sessions table

In [63]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessions (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[8], line[3], line[0], line[9], line[5]))

#### Step 3: Do a SELECT query to get artist, song_title, song_length during session_id='338' & item_in_session = '4'

In [64]:
query = "select artist, song_title, song_length from sessions WHERE session_id='338' AND item_in_session = '4'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Step 4: Add result of the above select query to a Dataframe and display

In [65]:
# Create and populate Dataframe

columns = ["artist", "song_title", "song_length"]
session_data = list()

for row in rows:
    session_data.append([row.artist, row.song_title, row.song_length])

df = pd.DataFrame(session_data, columns=columns)
df.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


# Task 2
    Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Step 1: Create user_history table

In [66]:
query = "CREATE TABLE IF NOT EXISTS user_history "
query = query + "(user_id text, session_id text, item_in_session text, artist text, song_title text, user text, \
PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Step 2: Get data from event_datafile_new.csv file and insert into user_history table

In [67]:
# Insert Data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history (user_id, session_id, item_in_session, artist, song_title, user)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        name = line[1] + ' ' + line[4]
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], name))

#### Step 3: Do a SELECT statement to get artist, song and user with a given userid and sessionid sorted by item_in_session

In [68]:
query = "select item_in_session, artist, song_title, user from user_history WHERE user_id='10' AND session_id = '182'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Step 4: Add result of the above select query to a Dataframe and display

In [69]:
columns = ["item_in_session", "artist", "song_title", "user"]
user_data = list()

# Sorted by item_in_session
for row in rows:
    user_data.append([row.item_in_session, row.artist, row.song_title, row.user])

df = pd.DataFrame(user_data, columns=columns, )
df.head()

,item_in_session,artist,song_title,user
0,0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,1,Three Drives,Greece 2000,Sylvie Cruz
2,2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


# Task 3
    Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Step 1: Create song_history Table

In [70]:
query = "CREATE TABLE IF NOT EXISTS song_history "
query = query + "(song_title text, user text, PRIMARY KEY (song_title, user))"
try:
    session.execute(query)
except Exception as e:
    print(e)     

#### Step 2: Get data from event_datafile_new.csv file and insert into song_history table

In [71]:
# Insert Data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (song_title, user)"
        query = query + "VALUES (%s, %s)"
        name = line[1] + ' ' + line[4]
        session.execute(query, (line[9], name))

#### Step 3: Do a SELECT statement to get user (first and last name) with the given song title

In [72]:
query = "select user from song_history WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


#### Step 4: Add result of all users who listened to the given song_title to a Dataframe and display

In [73]:
columns = ["user"]
song_data = list()

for row in rows:
    song_data.append([row.user])

df = pd.DataFrame(song_data, columns=columns, )
df.head()

,user
0,Jacqueline Lynch
1,Sara Johnson
2,Tegan Levine


### Drop the tables before closing out the sessions

In [74]:
tables = ["sessions", "user_history", "song_history"]

for table in tables:
    query = f"drop table {table}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [75]:
session.shutdown()
cluster.shutdown()